# Parsing and Cleaning PHEME RNR Dataset Events

In [86]:
# Load dependencies for this Jupyter Notebook
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.decomposition import PCA,SparsePCA,KernelPCA
from sklearn.manifold import TSNE, Isomap
import matplotlib.pyplot as plt

#Train and Test preprocessing
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#Classifiers:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

## Parsing and Cleaning Data
This step takes the raw PHEME rumor dataset and saves it tabular format as CSV file. The original PHEME dataset consists of JSON files organized into directories by event and category (rumor or non-rumor). These three functions below parse the data, save it as a CSV file (if necessary), and load it into this notebook as a Pandas DataFrame from the "cached" CSV file.

### The Ottawa Shooting

In [3]:
ottawashooting = pd.read_csv("data/pheme-rnr-dataset/ottawashooting.csv",
                             dtype={"user.verified": np.bool,
                                    "user.default_profile": np.bool,
                                    "user.geo_enabled": np.bool},
                             engine="python")  # This engine helps when loading large datasets

# Remove rows with boolean False values in the hashtags_count column
ottawashooting = ottawashooting.drop(ottawashooting[ottawashooting.hashtags_count == "False"].index)
ottawashooting.hashtags_count = ottawashooting.hashtags_count.astype(np.int32)

# Remove the row with NoneType in the retweet_count column
ottawashooting = ottawashooting.drop(ottawashooting[ottawashooting.retweet_count.isnull()].index)
ottawashooting["retweet_count"] = ottawashooting["retweet_count"].astype(np.int32)

# Convert datetime strings to timestamps
for col in ["user.created_at", "created"]:
    ottawashooting[col] = pd.to_datetime(ottawashooting[col]).astype(np.int64) / 1e6
    
ottawashooting.head()

,coordinates,created,favorite_count,has_url,hashtags_count,id,in_reply_id,in_reply_user,is_rumor,is_source_tweet,...,user.followers_count,user.friends_count,user.geo_enabled,user.listed_count,user.location,user.profile_users_background_image,user.time_zone,user.tweets_count,user.utf_offset,user.verified
0,NaN,1.413986e+12,21,False,2,524922078638903296,NaN,NaN,False,True,...,66724.0,296.0,True,1661.0,Ottawa,True,Eastern Time (US & Canada),38424.0,-14400.0,True
1,NaN,1.413986e+12,0,False,0,524922392582586368,524922078638903296,NaN,False,False,...,278.0,636.0,True,3.0,Tallon IV,True,Eastern Time (US & Canada),4116.0,-14400.0,True
2,NaN,1.413986e+12,0,False,0,524922589861658624,524922392582586368,NaN,False,False,...,4904.0,2026.0,True,209.0,MontrÃ©al,True,Eastern Time (US & Canada),19304.0,-14400.0,True
3,NaN,1.413986e+12,0,False,0,524922681595269120,524922078638903296,NaN,False,False,...,3646.0,2203.0,True,62.0,analystepolice@yahoo.ca,True,Central Time (US & Canada),16771.0,-18000.0,True
4,NaN,1.413986e+12,0,False,0,524923071090917376,524922681595269120,NaN,False,False,...,178.0,130.0,True,5.0,"MontrÃ©al,Lachine",True,Quito,8047.0,-18000.0,True


## Aggregating Thread-level Data

**Bold features** represent high performing features identified in C. Buntain and J. Golbeck, ["Automatically Identifying Fake News in Popular Twitter Threads"](http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8118443&isnumber=8118402)

1. `favorite_total`: The sum of all favorites from all tweets in this thread.
1. `retweet_total`: The sum of all retweets of all tweets in this thread.
1. `is_rumor`: Was this thread classified as rumor? Either `True` or `False`.
1. `url_proportion`: The fraction of tweets in a thread with a URL in the tweet text. Ranges from $[0,1]$.
1. `thread_length`: The number of tweets in this thread.
1. **`hashtag_proportion`**: The fraction of tweets in a thread with a hashtag in the tweet text. Ranges from $[0,1]$.
1. **`smile_emoji_proportion`**: The fraction of tweets in a thread with a smile emoji (😊) in the tweet text. Ranges from $[0,1]$.
1. **`src_followers_count`**: The number of followers of the original poster of the thread.
1. *`src_list_count`*: The number of lists the user has currated or the number of lists the user is a member of, but not sure which one.
1. `is_src_verified`: Is the source user's Twitter account verified? Either `True` or `False`.
1. `symbol_total`: The total number of $cashtags, a.k.a [symbols](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/entities-object.html#symbols), in a tweet thread.
1. `src_tweets_total`: The total number of tweets authored by the user of the source tweet.
1. `src_user_creation_date`: The timestamp of when the user of the source tweet first created a Twitter account.
1. `first_resp`: The timestamp of the first response to the source tweet.
1. `last_resp`: The timestamp of the last response to the source tweet
1. `resp_var`: The variance in the timestamps of responses to the source tweet
1. `src_user_geo_enabled`: Has the source [user of the tweet](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object.html) enabled the possibility of geotagging their tweets?
1. `geo_enabled_proportion`: Proportion of users in a tweet thread who have enabled the possibility of geotagging their tweets.

In [4]:
def agg_event_data(df, limit=0):
    """ Aggregate tabular tweet data from a PHEME event into aggregated thread-level data
    
    Params:
        - df: the DataFrame with tabular tweet data
       
    Return: A DataFrame with thread-level data for this event
    """
    data = df.head(limit) if limit > 0 else df
    data = data.replace({"has_url": {"True": True, "False": False}})
    
    # Returns the proportion of True values in col
    bool_prop = lambda col : np.sum(col) / len(col)
    
    agg = data.groupby("thread") \
        .agg({"favorite_count": sum,
              "retweet_count": sum,
              "is_rumor": max,
              "has_url": bool_prop,
              "id": len,
              "user.geo_enabled": bool_prop,
              "symbols_count": sum,
              "user.default_profile": bool_prop,
              "hashtags_count": lambda col: len([True for total in col if total > 0]) / len(col),
              "text": lambda col: len([True for txt in col if "😊" in txt]) / len(col)}) \
        .rename(columns={"favorite_count": "favorite_total",
                         "retweet_count": "retweet_total",
                         "user.friends_count": "friends_total",
                         "id": "thread_length",
                         "has_url":"url_proportion",
                         "hashtags_count": "hashtag_proportion",
                         "text": "smile_emoji_proportion",
                         "user.default_profile": "default_profile_proportion",
                         "user.geo_enabled": "geo_enabled_proportion",
                         "symbols_count": "symbol_total"})
    
    # source tweets will have equal thread id and tweet id
    src = data[data["thread"] == data["id"]][["thread", 
                                              "user.followers_count", 
                                              "user.listed_count",
                                              "user.verified",
                                              "created",
                                              "user.geo_enabled",
                                              "user.created_at",
                                              "user.tweets_count"]] \
        .rename(columns={"user.followers_count": "src_followers_count",
                         "user.listed_count": "src_listed_count",
                         "user.verified": "src_user_verified",
                         "user.created_at": "src_user_creation_date",
                         "user.geo_enabled": "src_user_geo_enabled",
                         "user.tweets_count": "src_tweets_total"})
    def f(x):
        d = []
        d.append(min(x["created"]))
        d.append(max(x["created"]))
        d.append(np.var(x["created"]))
        return pd.Series(d, index=["first_resp", "last_resp","resp_var"])
    
    non_src = data[data["thread"] != data["id"]] \
        .groupby("thread") \
        .apply(f)

    dfs = [agg, src, non_src]
    
    thrd_data = reduce(lambda left,right: pd.merge(left,right, on="thread"), dfs)
    
    thrd_data["first_resp"]=thrd_data["first_resp"]-thrd_data["created"]
    thrd_data["last_resp"]=thrd_data["last_resp"]-thrd_data["created"]
    

    
    return thrd_data

In [5]:
ottawa_threads = agg_event_data(ottawashooting)

# Save CSV file in /data directory
ottawa_threads.to_csv("data/pheme-rnr-dataset/ottawashooting_threads.csv")

# import math
# Print the first five rows
ottawa_threads.head()
ottawa_threads_rumortags=ottawa_threads["is_rumor"]
ottawa_threads=ottawa_threads.drop(['is_rumor'],axis=1)

## Running a PCA


### used functions:

In [6]:
def myprint(*obj,time=10):
    if myprint.t<time:
        print(*obj)
def convertTrueFalseTo01(X):
    X[X==True]=1
    X[X==False]=0
    X[X=='False']=0
    X[X=='True']=1
    return X

    
def standardize_cols(X, mu=None, sigma=None):
    # Standardize each column with mean 0 and variance 1
    n_rows, n_cols = X.shape

    if mu is None:
        mu = np.mean(X, axis=0)

    if sigma is None:
        sigma = np.std(X, axis=0)
        sigma[sigma < 1e-8] = 1.

    return (X - mu) / sigma, mu, sigma


### Data Preprocessing:

In [40]:
ottawa_threads_values=ottawa_threads.values
n,d=ottawa_threads_values.shape
ottawa_threads_values=convertTrueFalseTo01(ottawa_threads_values[:,1:d])
n,d=ottawa_threads_values.shape

ottawa_threads_rumortags_values=convertTrueFalseTo01(ottawa_threads_rumortags.values)

ottawa_threads_values,_,_=standardize_cols(ottawa_threads_values.astype(float))

n,d=ottawa_threads_values.shape
print(ottawa_threads_values.shape)

(857, 19)


### PCA:

In [41]:
model=PCA(n_components=2)
model.fit(ottawa_threads_values)
Z_PCA=model.transform(ottawa_threads_values)
plt.figure()
plt.title("PCA")
plt.scatter(Z_PCA[:,0],Z_PCA[:,1],c=ottawa_threads_rumortags_values)
plt.show()

### TSNE:

In [8]:
model=TSNE(n_components=2)
Z_TSNE=model.fit_transform(ottawa_threads_values)
plt.figure()
plt.title("TSNE")
plt.scatter(Z_TSNE[:,0],Z_TSNE[:,1],c=ottawa_threads_rumortags_values)
plt.show()

### Isomap:

In [112]:
model=Isomap(n_neighbors=5, n_components=2)
Z_Isomap=model.fit_transform(ottawa_threads_values)
plt.figure()
plt.title("Isomap")
plt.scatter(Z_Isomap[:,0],Z_Isomap[:,1],c=ottawa_threads_rumortags_values)
plt.show()

### SparsePCA:

In [117]:
model=SparsePCA(n_components=2,normalize_components=True)
model.fit(ottawa_threads_values)
Z_PCA=model.transform(ottawa_threads_values)
plt.figure()
plt.title("SparsePCA")
plt.scatter(Z_PCA[:,0],Z_PCA[:,1],c=ottawa_threads_rumortags_values)
plt.show()

### KernelPCA:

In [120]:
model=KernelPCA(n_components=2)
model.fit(ottawa_threads_values)
Z_PCA=model.transform(ottawa_threads_values)
plt.figure()
plt.title("KernelPCA")
plt.scatter(Z_PCA[:,0],Z_PCA[:,1],c=ottawa_threads_rumortags_values)
plt.show()

## Running some classifiers
### Train and Test data separation:

In [38]:
X_train, X_test, y_train, y_test = train_test_split(ottawa_threads_values, ottawa_threads_rumortags_values, test_size=0.25, random_state=45)
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train=le.transform(y_train)
y_test=le.transform(y_test)
print(X_train.shape,X_test.shape,y_train.shape)
print('y_train bincount:', np.bincount(y_train)/np.sum(np.bincount(y_train)))
print('y_test bincount:', np.bincount(y_test)/np.sum(np.bincount(y_test)))

(642, 19) (215, 19) (642,)
y_train bincount: [0.46573209 0.53426791]
y_test bincount: [0.46976744 0.53023256]


### SVM.SVC:

In [56]:
def test_model(model):
    model.fit(X_train,y_train)
    y_test_hat=model.predict(X_test)
    print('train error:', np.mean(model.predict(X_train)==y_train))
    print('test error:', np.mean(y_test_hat==y_test))

In [58]:
clf = svm.SVC(gamma='scale', kernel='linear', C=1)
test_model(model)

train error: 0.7601246105919003
test error: 0.6976744186046512


In [63]:
model = KNeighborsClassifier(n_neighbors=5)
test_model(model)

train error: 0.7601246105919003
test error: 0.6976744186046512


In [68]:
model=DecisionTreeClassifier(random_state=0)
test_model(model)

train error: 1.0
test error: 0.6511627906976745


In [93]:
model=RandomForestClassifier(n_estimators=100, max_depth=4, random_state=4, max_features=2)
test_model(model)

train error: 0.7741433021806854
test error: 0.6976744186046512


In [81]:
model=AdaBoostClassifier(n_estimators=100)
test_model(model)

train error: 0.8551401869158879
test error: 0.6744186046511628


In [87]:
model=GaussianProcessClassifier(1.0 * RBF(1.0))
test_model(model)

train error: 0.719626168224299
test error: 0.6465116279069767
